# Data Processing

In [1]:
from pathlib import Path
import pandas as pd


DATA_PATH = Path('../data')
artists_df = pd.read_csv(DATA_PATH/'artists.csv', index_col=0)

songs_df = pd.read_csv(DATA_PATH/'songs.csv', index_col=0)
streams_df = pd.read_csv(DATA_PATH/'streams.csv', index_col=0)
users_df = pd.read_csv(DATA_PATH/'users.csv', index_col=0)
weeks = streams_df.week_no.max() + 1
weeks

30

In [2]:
if weeks == 30:
    MINIMAL_HISTORY_WEEKS = 5
    PREDICT_IN_FUTURE_WEEKS = 5
    MAX_HISTORY_WEEK = 29
elif weeks == 100:
    MINIMAL_HISTORY_WEEKS = 15 
    PREDICT_IN_FUTURE_WEEKS = 20
    MAX_HISTORY_WEEK = 99
else:
    raise(Exception(f'You should prepare a config for {weeks} weeks'))

In [3]:
from tqdm import tqdm

results = []

small_sample_songs_df = songs_df
# .loc[:1000]

for song_id, song in tqdm(small_sample_songs_df.iterrows(), total=small_sample_songs_df.shape[0]):
    for week_no in range(song.week_released + MINIMAL_HISTORY_WEEKS, MAX_HISTORY_WEEK-PREDICT_IN_FUTURE_WEEKS+1):
        # print(week_released)
        sel_artist_df = artists_df.loc[song.artist_id]
        sel_streams_df = streams_df[
            (streams_df.song_id==song_id) &
            (streams_df.week_no<week_no) & 
            (streams_df.week_no>=week_no-MINIMAL_HISTORY_WEEKS) 
        ]
        sel_users_df = users_df.loc[sel_streams_df.user_id.unique()]
        
        perc_listeners_from_continent_dict = (pd.get_dummies(sel_users_df.continent, prefix='perc_listeners_from').sum()/sel_users_df.shape[0]).to_dict()
        sel_target_streams_df = streams_df[
            (streams_df.song_id==song_id) &
            (streams_df.week_no==week_no+PREDICT_IN_FUTURE_WEEKS)
        ]
        
        row = {
            "genre": song.genre,
            "is_premium": song.is_premium,
            "artist_continent": sel_artist_df.continent,
            "weeks_after_artist_created": week_no - sel_artist_df.week_no_created,
            "weeks_after_song_released": week_no - song.week_released,
            "unique_listeners": sel_streams_df.user_id.nunique(),
            "total_streams": sel_streams_df.shape[0],
            "perc_of_subscribed_listeners": 0 if sel_users_df.empty else sel_users_df.is_subscribed.mean(),
            "average_listener_age": 0 if sel_users_df.empty else sel_users_df.age.mean(),
            **perc_listeners_from_continent_dict,
            'target': sel_target_streams_df.shape[0],
        }
        results.append(row)
        
df = pd.DataFrame(results)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 302/302 [00:04<00:00, 61.92it/s]


In [5]:
df = pd.DataFrame(results)

In [6]:
print(df)

          genre  is_premium artist_continent  weeks_after_artist_created  \
0    Electronic       False    North America                           6   
1    Electronic       False    North America                           7   
2    Electronic       False    North America                           8   
3    Electronic       False    North America                           9   
4    Electronic       False    North America                          10   
..          ...         ...              ...                         ...   
731        Rock       False    North America                           9   
732        Jazz        True    North America                           7   
733        Rock       False           Oceana                           6   
734         Pop       False           Oceana                           5   
735         Pop       False    South America                           5   

     weeks_after_song_released  unique_listeners  total_streams  \
0                   

In [7]:
df = df.join(pd.get_dummies(df.genre, prefix='song_genre').fillna(0))
df = df.join(pd.get_dummies(df.artist_continent, prefix='artist_continent').fillna(0))
df = df.drop(columns=['genre', 'artist_continent'], errors='ignore')
df.is_premium = df.is_premium.astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   is_premium                         736 non-null    int32  
 1   weeks_after_artist_created         736 non-null    int64  
 2   weeks_after_song_released          736 non-null    int64  
 3   unique_listeners                   736 non-null    int64  
 4   total_streams                      736 non-null    int64  
 5   perc_of_subscribed_listeners       736 non-null    float64
 6   average_listener_age               736 non-null    float64
 7   perc_listeners_from_Africa         726 non-null    float64
 8   perc_listeners_from_Asia           734 non-null    float64
 9   perc_listeners_from_Europe         736 non-null    float64
 10  perc_listeners_from_North America  736 non-null    float64
 11  perc_listeners_from_Oceana         516 non-null    float64

In [8]:
df.loc[:, df.columns.str.startswith('perc_listeners_from')] = df.loc[:, df.columns.str.startswith('perc_listeners_from')].fillna(0)

In [9]:
df.to_csv(DATA_PATH/f'train_data_{weeks}_weeks.csv')